# XCH4 dataset satellite lev2

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import flox.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define request

In [ ]:
# Time parameters
year_start = 2005
year_stop = 2006

collection_id = "satellite-methane"
request = {
    "processing_level": "level_2",
    "variable": "xch4",
    "sensor_and_algorithm": "merged_emma",
    "version": "4.4",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{i:02d}" for i in range(1, 13)],
    "day": [f"{i:02d}" for i in range(1, 32)],
}

## Define function to cache

In [ ]:
def arithmetic_unweighted_average(ds, d_lon, d_lat, lon1):
    if lon1 == 180:
        lon_lims = [-180, 180]
    elif lon1 == 360:
        lon_lims = [0, 360]
    else:
        raise ValueError(f"lon1 must be 180 or 360. {lon1=}")

    lon_groups = np.arange(lon_lims[0], lon_lims[1] + d_lon, d_lon)
    lon_groups[0] -= d_lon
    lat_groups = np.arange(-90, 90 + d_lat, d_lat)
    lat_groups[0] -= d_lat

    ds = flox.xarray.xarray_reduce(
        ds,
        "latitude",
        "longitude",
        func="mean",
        expected_groups=(
            pd.IntervalIndex.from_breaks(lat_groups),
            pd.IntervalIndex.from_breaks(lon_groups),
        ),
        keep_attrs=True,
    )
    ds = ds.rename(longitude_bins="longitude", latitude_bins="latitude")
    ds = ds.assign_coords(
        longitude=np.arange(lon_lims[0] + d_lon / 2, lon_lims[1] + d_lon / 2, d_lon),
        latitude=np.arange(-90 + d_lat / 2, 90 + d_lat / 2, d_lat),
    )
    return ds


def monthly_regrid(ds, d_lon, d_lat, lon1=180):
    """Resample to monthly regular grid.

    Parameters
    ----------
    ds: Dataset
        Dataset to resample
    d_lon, d_lat: int
        Longitude/latitude step size
    lon1: int, {180, 360}
        Right longitude bound. According to which convention is used,
        longitudes will vary from -180 to 180 or from 0 to 360.

    Returns
    -------
    Dataset
    """
    ds = ds.set_coords(["longitude", "latitude"])
    ds_out = ds.resample(time="1MS").map(
        arithmetic_unweighted_average, d_lon=d_lon, d_lat=d_lat, lon1=lon1
    )
    ds_out["time"].attrs["standard_name"] = "time"
    return ds_out

## Download and transform data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"year": 1},
    transform_func=monthly_regrid,
    transform_func_kwargs={"d_lon": 1, "d_lat": 1, "lon1": 180},
)

## Compute seasonal anomalies

In [ ]:
climatology = diagnostics.seasonal_weighted_mean(ds)
with xr.set_options(keep_attrs=True):
    anomaly = (
        ds.groupby("time.year").map(diagnostics.seasonal_weighted_mean) - climatology
    )
for da in anomaly.data_vars.values():
    if "long_name" in da.attrs:
        da.attrs["long_name"] += " Anomaly"

## Plot seasonal anomalies

In [ ]:
plot.projected_map(
    anomaly["xch4"], projection=ccrs.Robinson(), col="season", row="year", robust=True
)